In [1]:
# Import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import faceBlendCommon as face


In [14]:
# Define the eye aspect ratio function
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # Compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # Return the eye aspect ratio
    return ear

# Define Eye Aspect Ratio threshold 
EYE_AR_THRESH = 0.52
EYE_AR_CONSEC_FRAMES = 3



# Initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_81_face_landmarks.dat")

# Grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

#vs = VideoStream(src=0).start()
#time.sleep(1.0)
frame = cv2.imread("/home/snehal/Downloads/1.jpg")
# Loop over frames from the video stream
while True:
    # If this is a file video stream, then we need to check if
    # there any more frames left in the buffer to process

    # Grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale
    # channels)
    #frame = vs.read()
    
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    rects = detector(gray, 0)

    # Loop over the face detections
    for rect in rects:
        # Determine the facial landmarks for the face region, then
        # Convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # Extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # Average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR)

        # Compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        # Check to see if the eye aspect ratio is below the EAR
        # threshold, if so, display eye status as Fully closed.
        if ear < EYE_AR_THRESH:
            cv2.putText(frame, "Eye: {}".format("Fully closed"), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            
        
        # Else if the EAR is between the closed and open threshold,display
        # eye status as partially open 
        elif( ear > EYE_AR_THRESH and ear < 0.61 ):
            cv2.putText(frame, "Eye: {}".format("half closed"), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    


        # Else, set the eye status as fully open.
        else:
            #im = frame
            im = cv2.imread("/home/snehal/Downloads/1.jpg")
            PREDICTOR_PATH = r"./shape_predictor_81_face_landmarks.dat"
            faceDetector = dlib.get_frontal_face_detector()
            landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)
            landmarks = face.getLandmarks(faceDetector, landmarkDetector, im)
            
            def createEyeMask(eyeLandmarks, im):
                leftEyePoints = eyeLandmarks
                eyeMask = np.zeros_like(im)
                cv2.fillConvexPoly(eyeMask, np.int32(leftEyePoints), (255, 255, 255))
                eyeMask = np.uint8(eyeMask)
                return eyeMask

            def findIris(eyeMask, im, thresh):
                r = im[:,:,2]
                _, binaryIm = cv2.threshold(r, thresh, 255, cv2.THRESH_BINARY_INV)
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4))
                morph = cv2.dilate(binaryIm, kernel, 1)
                morph = cv2.merge((morph, morph, morph))
                morph = morph.astype(float)/255
                eyeMask = eyeMask.astype(float)/255
                iris = cv2.multiply(eyeMask, morph)
                return iris

            def findCentroid(iris):
                M = cv2.moments(iris[:,:,0])
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                centroid = (cX,cY)
                return centroid

            def createIrisMask(iris, centroid):
                cnts, _ = cv2.findContours(np.uint8(iris[:,:,0]), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                flag = 10000
                final_cnt = None
                for cnt in cnts:
                    (x,y),radius = cv2.minEnclosingCircle(cnt)
                    distance = abs(centroid[0]-x)+abs(centroid[1]-y)
                    if distance < flag :
                        flag = distance
                        final_cnt = cnt
                    else:
                        continue
                (x,y),radius = cv2.minEnclosingCircle(final_cnt)
                center = (int(x),int(y))
                radius = int(radius) - 2

                irisMask = np.zeros_like(iris)
                inverseIrisMask = np.ones_like(iris)*255
                cv2.circle(irisMask,center,radius,(255, 255, 255),-1)
                cv2.circle(inverseIrisMask,center,radius,(0, 0, 0),-1)
                irisMask = cv2.GaussianBlur(irisMask, (5,5), cv2.BORDER_DEFAULT)
                inverseIrisMask = cv2.GaussianBlur(inverseIrisMask, (5,5), cv2.BORDER_DEFAULT)

                return irisMask, inverseIrisMask

            def changeEyeColor(im, irisMask, inverseIrisMask):
                imCopy = cv2.applyColorMap(im, cv2.COLORMAP_AUTUMN)
                imCopy = imCopy.astype(float)/255
                irisMask = irisMask.astype(float)/255
                inverseIrisMask = inverseIrisMask.astype(float)/255
                im = im.astype(float)/255
                faceWithoutEye = cv2.multiply(inverseIrisMask, im)
                newIris = cv2.multiply(irisMask, imCopy)
                result = faceWithoutEye + newIris
                return result

            def float642Uint8(im):
                im2Convert = im.astype(np.float64) / np.amax(im)
                im2Convert = 255 * im2Convert 
                convertedIm = im2Convert.astype(np.uint8)
                return convertedIm

            # Create eye mask using eye landmarks from facial landmark detection
            leftEyeMask = createEyeMask(landmarks[37:42], im)
            rightEyeMask = createEyeMask(landmarks[43:49], im)

            # Find the iris by thresholding the red channel of the image within the boundaries of the eye mask
            leftIris = findIris(leftEyeMask, im, 39)
            rightIris = findIris(rightEyeMask, im, 49)

            # Find the centroid of the binary image of the eye
            leftIrisCentroid = findCentroid(leftIris)
            rightIrisCentroid = findCentroid(rightIris)

            # Generate the iris mask and its inverse mask
            leftIrisMask, leftInverseIrisMask = createIrisMask(leftIris, leftIrisCentroid)
            rightIrisMask, rightInverseIrisMask = createIrisMask(rightIris, rightIrisCentroid)


            # Change the eye color and merge it to the original image
            coloredEyes = changeEyeColor(im, rightIrisMask, rightInverseIrisMask)
            coloredEyes = float642Uint8(coloredEyes)
            coloredEyes = changeEyeColor(coloredEyes, leftIrisMask, leftInverseIrisMask)

            # Present results
            cv2.imshow("", coloredEyes)
            
            cv2.putText(frame, "Eye: {}".format("Fully open - Iris Detected"), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            #cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    
    

    # Show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If the `q` key is pressed, break from the loop
    if key == ord("q"):
        break

# Cleanup
cv2.destroyAllWindows()
#vs.stop()

[INFO] loading facial landmark predictor...
